In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model_16")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=50,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=1000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=5,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model_16', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018BCA49DBA8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

INFO:tensorflow:global_step/sec: 6.19513
INFO:tensorflow:probabilities = [[0.09649971 0.09826566 0.10544965 0.09222008 0.09131997 0.11204943
  0.0932181  0.11599628 0.09132864 0.10365245]
 [0.10347097 0.08851179 0.10494371 0.09820484 0.10578709 0.10825955
  0.08623647 0.11708754 0.09575482 0.09174324]
 [0.09448459 0.09167907 0.09470522 0.11129263 0.10132904 0.09701726
  0.09071919 0.1089443  0.10722216 0.10260657]
 [0.10911474 0.0829841  0.10120209 0.11871196 0.09175321 0.0993224
  0.10280134 0.10262936 0.09616452 0.09531621]
 [0.09545162 0.07982145 0.09723537 0.1104011  0.1068995  0.09442437
  0.09560661 0.11465942 0.10366111 0.10183947]
 [0.09540354 0.08747053 0.10830557 0.10608907 0.10011628 0.09352271
  0.09522536 0.11451682 0.10004467 0.09930545]
 [0.09823077 0.09331622 0.10651053 0.1060454  0.09585574 0.10443936
  0.09311406 0.11304384 0.09414112 0.09530304]
 [0.10471565 0.09239195 0.10499725 0.10112751 0.10980291 0.10274439
  0.09038496 0.11578391 0.08518026 0.09287125]
 [0.1027

INFO:tensorflow:global_step/sec: 6.3346
INFO:tensorflow:probabilities = [[0.09807547 0.08970343 0.10196818 0.09262507 0.09843923 0.10517189
  0.09080268 0.11736428 0.10093208 0.10491773]
 [0.10252599 0.08073919 0.10200134 0.0935159  0.1037541  0.10437107
  0.1004907  0.11262334 0.10726511 0.09271322]
 [0.09831989 0.08007162 0.10695257 0.09551533 0.10213115 0.09114186
  0.1188745  0.10749141 0.11828338 0.08121823]
 [0.09939591 0.08518379 0.10430195 0.09499835 0.10874882 0.09423368
  0.09361879 0.12048276 0.10655821 0.09247784]
 [0.10549046 0.07516424 0.10141779 0.09526232 0.11280674 0.09744348
  0.10552552 0.10239786 0.11183988 0.09265164]
 [0.09999103 0.09179544 0.09839193 0.09457874 0.10111254 0.10515063
  0.10578951 0.10332719 0.10980106 0.09006192]
 [0.09126976 0.07353509 0.10273086 0.11691853 0.11140813 0.10875004
  0.09710636 0.1115734  0.10190834 0.08479952]
 [0.10473613 0.07668429 0.10608437 0.1076588  0.09337156 0.10854575
  0.10867889 0.11006228 0.0933382  0.09083971]
 [0.1144

INFO:tensorflow:global_step/sec: 6.17578
INFO:tensorflow:probabilities = [[0.10936157 0.07738914 0.10086276 0.10384969 0.09854487 0.09187739
  0.10163237 0.11325718 0.1019148  0.10131028]
 [0.10416549 0.0812512  0.09430318 0.10198089 0.09841961 0.10288153
  0.09363081 0.1178345  0.11000522 0.09552758]
 [0.09316047 0.09799111 0.1047423  0.10379612 0.09953547 0.09747943
  0.09060422 0.10866597 0.102497   0.10152791]
 [0.11953932 0.08223711 0.09336131 0.0865033  0.10963336 0.09926813
  0.09574044 0.11129525 0.10504308 0.09737869]
 [0.09937418 0.07810657 0.10920991 0.10182595 0.09414184 0.08752442
  0.10264971 0.11294941 0.11043584 0.10378215]
 [0.11138076 0.07401306 0.10055765 0.10494837 0.08145545 0.1171482
  0.0967121  0.12541312 0.09681623 0.09155508]
 [0.08790013 0.09809927 0.10236382 0.10380033 0.09331805 0.10510018
  0.09909344 0.10724147 0.11042912 0.09265426]
 [0.11133498 0.07969517 0.09421501 0.11034714 0.10314075 0.095833
  0.10861525 0.10897335 0.09854977 0.08929557]
 [0.108299

INFO:tensorflow:global_step/sec: 6.29527
INFO:tensorflow:probabilities = [[0.09165849 0.08444278 0.10110828 0.10677744 0.11052151 0.10319903
  0.09417108 0.10880301 0.09756266 0.10175577]
 [0.1063934  0.07506648 0.09659847 0.08770058 0.10946332 0.10364351
  0.09872745 0.10272675 0.10548032 0.11419974]
 [0.12171952 0.06888158 0.08962614 0.11436755 0.11668873 0.10855407
  0.10688725 0.10870276 0.08555003 0.07902247]
 [0.10100982 0.08435983 0.09335586 0.11824153 0.10756451 0.09866389
  0.09139357 0.10567503 0.09394449 0.10579143]
 [0.10364868 0.08713457 0.09652749 0.10835128 0.09128235 0.0994579
  0.10151514 0.12338036 0.09663554 0.09206667]
 [0.11047272 0.07670114 0.13165964 0.1118936  0.09940577 0.09565514
  0.08792819 0.10820097 0.0892431  0.08883971]
 [0.10002139 0.08319032 0.10571418 0.11408876 0.09335104 0.08851276
  0.09342419 0.11152679 0.10959225 0.10057825]
 [0.106401   0.0805942  0.09476873 0.10032667 0.11380477 0.09432331
  0.10243611 0.1122504  0.1014645  0.09363025]
 [0.0991

...

INFO:tensorflow:global_step/sec: 6.03908
INFO:tensorflow:probabilities = [[0.10077187 0.06995376 0.09517629 0.08420327 0.11391532 0.0940816
  0.09782202 0.10956505 0.12521796 0.10929283]
 [0.1044078  0.07591156 0.11414605 0.10932793 0.10967647 0.0907406
  0.12644692 0.09439578 0.08946209 0.08548485]
 [0.09746182 0.07331745 0.09688826 0.10963085 0.11287177 0.09520305
  0.10133231 0.09632862 0.11131486 0.10565095]
 [0.09145109 0.09543037 0.10328201 0.11166351 0.093122   0.08561277
  0.10658331 0.08952661 0.10006533 0.12326298]
 [0.11349642 0.07399518 0.10485907 0.10788093 0.10503872 0.11605839
  0.0882704  0.09586408 0.104719   0.08981778]
 [0.12710382 0.06305086 0.10550146 0.1114021  0.07876879 0.07736337
  0.09439154 0.11676778 0.13906968 0.08658052]
 [0.08147004 0.09500485 0.09513043 0.10829013 0.09455704 0.09452419
  0.09214749 0.13228883 0.1031424  0.10344461]
 [0.07919723 0.11184461 0.09749088 0.11654656 0.1044609  0.08544691
  0.09246756 0.11010209 0.10546403 0.0969792 ]
 [0.08566

INFO:tensorflow:global_step/sec: 6.19424
INFO:tensorflow:probabilities = [[0.09560098 0.06172411 0.0848337  0.20817311 0.07729849 0.10447051
  0.09457731 0.0901143  0.09575146 0.08745603]
 [0.08462918 0.18118134 0.08632915 0.11250386 0.07885858 0.07266679
  0.09239871 0.0957736  0.10597616 0.08968256]
 [0.07835422 0.11534516 0.09059888 0.10589522 0.08291381 0.10362267
  0.08131845 0.10626514 0.13231274 0.10337371]
 [0.10583008 0.06054367 0.12586689 0.10627525 0.13609499 0.0734062
  0.07880425 0.10429453 0.11577681 0.09310737]
 [0.14616016 0.05145313 0.09557468 0.107306   0.10496686 0.09911621
  0.11031656 0.11824847 0.09029443 0.07656354]
 [0.10436535 0.06850143 0.08024675 0.08934135 0.10308968 0.10108689
  0.07805919 0.11030212 0.11440065 0.15060659]
 [0.0736964  0.08586091 0.10067742 0.1063953  0.10217835 0.08757143
  0.08782282 0.10010021 0.12940387 0.12629335]
 [0.08010095 0.10872584 0.10763554 0.08995689 0.09151766 0.09629175
  0.10480504 0.10979097 0.12429356 0.08688178]
 [0.1037

INFO:tensorflow:Saving checkpoints for 1000 into /tmp/mnist_convnet_model_16\model.ckpt.
INFO:tensorflow:Loss for final step: 1.8672994.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-10-00:22:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model_16\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-10-00:22:55
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7482, global_step = 1000, loss = 1.9228528
{'accuracy': 0.7482, 'loss': 1.9228528, 'global_step': 1000}


SystemExit: 

c:\users\ninad\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
